In [1]:
# pip install optuna!

In [4]:
# Training Notebook: Retrain CatBoost with Optuna tuning
# Save as train_aqi_optuna.ipynb / train_aqi_optuna.py

import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from catboost import CatBoostRegressor, Pool
import optuna
import joblib
import matplotlib.pyplot as plt

# ----------------- CONFIG -----------------
DATA_PATH = "Preprocessed_Data_Final.xls"   # your file
TARGET_COL = "AQI"
FEATURES_SAVE = "feature_cols_new.pkl"
MODEL_SAVE = "CatBoost_Optimized.pkl"

RANDOM_SEED = 42
N_TRIALS = 50          # change to 100+ for better tuning (longer)
TS_SPLITS = 5          # number of time-splits for CV
EARLY_STOPPING_ROUNDS = 100
VERBOSE = 100

np.random.seed(RANDOM_SEED)

# ----------------- Utility: load data -----------------
def load_data(path=DATA_PATH):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Training file not found: {path}")
    # Try csv first then excel
    try:
        df = pd.read_csv(path, parse_dates=["Datetime"])
    except Exception:
        df = pd.read_excel(path, engine="openpyxl")
    # Ensure Datetime column
    if "Datetime" in df.columns:
        df["Datetime"] = pd.to_datetime(df["Datetime"])
    else:
        raise ValueError("Data must have a 'Datetime' column.")
    return df

# ----------------- Define expected features -----------------
EXPECTED_FEATURES = [
    'StationId_enc','PM2.5','NO2','CO','SO2','O3','Temperature','DewPoint',
    'WindDirection','WindSpeed','Pressure','RelativeHumidity',
    'PM25_lag1','PM25_lag3','PM25_lag6','PM25_lag24',
    'NO2_lag1','O3_lag1',
    'PM25_roll3','PM25_roll6','PM25_roll12','PM25_roll24',
    'Hour_sin','Hour_cos',
    'Month_sin','Month_cos',
    'Pollution_Load','PM_Ratio','Temp_Humidity_Interaction','Wind_Inv',
    'Hour','DayOfWeek','IsWeekend','Month','Quarter','DayOfYear','Year',
    'Day_Monday','Day_Saturday','Day_Sunday','Day_Thursday','Day_Tuesday','Day_Wednesday',
    'Season_PostMonsoon','Season_Summer','Season_Winter'
]

# ----------------- Feature engineering helpers -----------------
def add_datetime_features(df):
    dt = df["Datetime"]
    df["Hour"] = dt.dt.hour
    df["DayOfWeek"] = dt.dt.weekday
    df["IsWeekend"] = (dt.dt.weekday >= 5).astype(int)
    df["Month"] = dt.dt.month
    df["Quarter"] = dt.dt.quarter
    df["DayOfYear"] = dt.dt.dayofyear
    df["Year"] = dt.dt.year
    df["Hour_sin"] = np.sin(2 * np.pi * df["Hour"] / 24)
    df["Hour_cos"] = np.cos(2 * np.pi * df["Hour"] / 24)
    df["Month_sin"] = np.sin(2 * np.pi * df["Month"] / 12)
    df["Month_cos"] = np.cos(2 * np.pi * df["Month"] / 12)

    # Day one-hot
    df["Day_Monday"] = (df["DayOfWeek"] == 0).astype(int)
    df["Day_Tuesday"] = (df["DayOfWeek"] == 1).astype(int)
    df["Day_Wednesday"] = (df["DayOfWeek"] == 2).astype(int)
    df["Day_Thursday"] = (df["DayOfWeek"] == 3).astype(int)
    df["Day_Friday"] = (df["DayOfWeek"] == 4).astype(int)
    df["Day_Saturday"] = (df["DayOfWeek"] == 5).astype(int)
    df["Day_Sunday"] = (df["DayOfWeek"] == 6).astype(int)

    # Seasons (approx)
    df["Season_Winter"] = df["Month"].isin([12,1,2]).astype(int)
    df["Season_Summer"] = df["Month"].isin([4,5,6]).astype(int)
    df["Season_PostMonsoon"] = df["Month"].isin([10,11]).astype(int)

    return df

def add_interactions(df):
    # safe casts
    df["PM2.5"] = pd.to_numeric(df.get("PM2.5", 0), errors="coerce").fillna(0)
    df["NO2"] = pd.to_numeric(df.get("NO2", 0), errors="coerce").fillna(0)
    df["O3"] = pd.to_numeric(df.get("O3", 0), errors="coerce").fillna(0)
    df["CO"] = pd.to_numeric(df.get("CO", 0), errors="coerce").fillna(0)
    df["SO2"] = pd.to_numeric(df.get("SO2", 0), errors="coerce").fillna(0)
    df["Temperature"] = pd.to_numeric(df.get("Temperature", 0), errors="coerce").fillna(0)
    df["RelativeHumidity"] = pd.to_numeric(df.get("RelativeHumidity", 0), errors="coerce").fillna(0)
    df["WindSpeed"] = pd.to_numeric(df.get("WindSpeed", 0), errors="coerce").fillna(0)

    df["Pollution_Load"] = df["PM2.5"] + df["NO2"] + df["CO"] + df["SO2"] + df["O3"]
    df["PM_Ratio"] = df["PM2.5"] / (df["NO2"] + df["O3"] + 1e-6)
    df["Temp_Humidity_Interaction"] = df["Temperature"] * df["RelativeHumidity"]
    df["Wind_Inv"] = 1.0 / (df["WindSpeed"].replace(0, np.nan) + 1e-6)
    df["Wind_Inv"] = df["Wind_Inv"].fillna(0)
    return df

def compute_lags_rolls(df, station_col="StationId", time_col="Datetime"):
    """
    Compute PM2.5 lags & rolls, NO2/O3 lag1 per station.
    Assumes df sorted by Datetime ascending.
    Works in-place and returns df.
    """
    df = df.sort_values([station_col, time_col])
    # We'll compute per-station
    lag_cols = ["PM25_lag1","PM25_lag3","PM25_lag6","PM25_lag24","NO2_lag1","O3_lag1"]
    roll_cols = ["PM25_roll3","PM25_roll6","PM25_roll12","PM25_roll24"]

    for sid, g in df.groupby(station_col):
        g = g.sort_values(time_col)
        pm = g["PM2.5"].astype(float).fillna(method="ffill").fillna(0)
        no2 = g["NO2"].astype(float).fillna(method="ffill").fillna(0)
        o3 = g["O3"].astype(float).fillna(method="ffill").fillna(0)

        idx = g.index

        # lags
        df.loc[idx, "PM25_lag1"] = pm.shift(1).values
        df.loc[idx, "PM25_lag3"] = pm.shift(3).values
        df.loc[idx, "PM25_lag6"] = pm.shift(6).values
        df.loc[idx, "PM25_lag24"] = pm.shift(24).values

        df.loc[idx, "NO2_lag1"] = no2.shift(1).values
        df.loc[idx, "O3_lag1"] = o3.shift(1).values

        # rolling means (min periods = 1 -> fallback)
        df.loc[idx, "PM25_roll3"] = pm.rolling(window=3, min_periods=1).mean().values
        df.loc[idx, "PM25_roll6"] = pm.rolling(window=6, min_periods=1).mean().values
        df.loc[idx, "PM25_roll12"] = pm.rolling(window=12, min_periods=1).mean().values
        df.loc[idx, "PM25_roll24"] = pm.rolling(window=24, min_periods=1).mean().values

    # fill remaining NaNs with sensible defaults
    for c in lag_cols + roll_cols:
        if c in df.columns:
            df[c] = df[c].astype(float).fillna(method="ffill").fillna(0)

    return df

In [5]:
# ----------------- Loading & preparing training data -----------------
df = load_data(DATA_PATH)
print("Raw data shape:", df.shape)
df = df.sort_values("Datetime").reset_index(drop=True)

# Ensure station encoding exists or create one
if "StationId_enc" not in df.columns:
    if "StationId" not in df.columns:
        raise ValueError("Training data must contain StationId or StationId_enc.")
    # create encoding
    df["StationId_enc"] = pd.factorize(df["StationId"])[0]

# ------- Ensure datetime & interactions -------
df = add_datetime_features(df)
df = add_interactions(df)

# ------- Compute lags & rolls -------
df = compute_lags_rolls(df, station_col="StationId", time_col="Datetime")

# ------- Ensure expected features exist (create if missing with fallbacks) -------
for feat in EXPECTED_FEATURES:
    if feat not in df.columns:
        # create safe fallback numeric column
        df[feat] = 0.0
        print(f"WARNING: Feature {feat} missing in source and created fallback zeros.")

# Drop rows where target is missing
df = df[~df[TARGET_COL].isna()].copy()
df = df.sort_values("Datetime").reset_index(drop=True)
print("Prepared data shape:", df.shape)

# ----------------- Feature list & X,y -----------------
feature_list = [f for f in EXPECTED_FEATURES if f in df.columns]
print(f"Using {len(feature_list)} features for training.")

X = df[feature_list].astype(float)
y = df[TARGET_COL].astype(float)



# ----------------- TimeSeriesSplit setup -----------------
tscv = TimeSeriesSplit(n_splits=TS_SPLITS)

# utility to evaluate predictions
def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred) ** 0.5  # manual sqrt
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "R2": r2}



Raw data shape: (1449833, 50)
Prepared data shape: (1449833, 51)
Using 46 features for training.


In [4]:

# ----------------- Baseline CatBoost (quick) -----------------
print("\nTraining baseline CatBoost (quick)...")
baseline_params = {
    "iterations": 1000,
    "learning_rate": 0.1,
    "depth": 6,
    "random_seed": RANDOM_SEED,
    "loss_function": "MAE",
    "verbose": 0,
    "early_stopping_rounds": 50
}

cv_maes = []
for train_idx, test_idx in tscv.split(X):
    X_tr, X_val = X.iloc[train_idx], X.iloc[test_idx]
    y_tr, y_val = y.iloc[train_idx], y.iloc[test_idx]

    model = CatBoostRegressor(**baseline_params)
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), verbose=False, use_best_model=True)
    pred = model.predict(X_val)
    cv_maes.append(mean_absolute_error(y_val, pred))

print("Baseline CV MAE (mean):", np.mean(cv_maes))

# ----------------- Optuna hyperparameter tuning -----------------
print("\nStarting Optuna tuning... (this may take time)")

def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 500, 3000, step=100),
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "random_strength": trial.suggest_float("random_strength", 0.0, 20.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.5),
    
        "loss_function": "MAE",
        "random_seed": RANDOM_SEED,
        "verbose": False,
    
        # GPU HERE
        "task_type": "GPU",
        "devices": "0"
    }

    # time-series CV -> compute mean MAE across splits
    maes = []
    for train_idx, test_idx in tscv.split(X):
        X_tr, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = CatBoostRegressor(**params)
        model.fit(X_tr, y_tr, eval_set=(X_val, y_val),
                  early_stopping_rounds=EARLY_STOPPING_ROUNDS, verbose=False, use_best_model=True)
        pred = model.predict(X_val)
        maes.append(mean_absolute_error(y_val, pred))

    return float(np.mean(maes))

study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

print("Best trial params:")
print(study.best_trial.params)
best_params = study.best_trial.params

Raw data shape: (1449833, 50)
Prepared data shape: (1449833, 51)
Using 46 features for training.

Training baseline CatBoost (quick)...


[I 2025-11-24 18:51:47,266] A new study created in memory with name: no-name-08baeb72-72e2-4d0f-a4b4-a1eab8de159e


Baseline CV MAE (mean): 24.74097526352017

Starting Optuna tuning... (this may take time)


  0%|          | 0/50 [00:00<?, ?it/s]

Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


...

Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


[I 2025-11-25 00:18:03,512] Trial 49 finished with value: 99.07945127056459 and parameters: {'iterations': 2400, 'depth': 7, 'learning_rate': 0.004972911883588379, 'l2_leaf_reg': 2.491572072806746, 'random_strength': 12.446766509544567, 'bagging_temperature': 0.5692463096472594}. Best is trial 21 with value: 25.537196106843133.
Best trial params:
{'iterations': 3000, 'depth': 8, 'learning_rate': 0.2987646259036188, 'l2_leaf_reg': 0.018949650626008688, 'random_strength': 16.993667922363702, 'bagging_temperature': 0.9398476987366979}

Training final model on full dataset...


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 103.1694105	total: 33.1ms	remaining: 1m 39s
100:	learn: 92.9906493	total: 3.23s	remaining: 1m 32s
200:	learn: 83.8974972	total: 6.39s	remaining: 1m 29s
300:	learn: 76.0529192	total: 9.56s	remaining: 1m 25s
400:	learn: 69.4540351	total: 12.6s	remaining: 1m 21s
500:	learn: 63.7415371	total: 15.9s	remaining: 1m 19s
600:	learn: 58.7515997	total: 19.1s	remaining: 1m 16s
700:	learn: 54.3575819	total: 22.2s	remaining: 1m 12s
800:	learn: 50.4233743	total: 25.2s	remaining: 1m 9s
900:	learn: 46.8417314	total: 28.4s	remaining: 1m 6s
1000:	learn: 43.5383220	total: 31.4s	remaining: 1m 2s
1100:	learn: 40.4385333	total: 34.3s	remaining: 59.2s
1200:	learn: 37.5931904	total: 37.2s	remaining: 55.7s
1300:	learn: 35.2972763	total: 40.2s	remaining: 52.5s
1400:	learn: 33.4677773	total: 43s	remaining: 49.1s
1500:	learn: 31.9973293	total: 45.9s	remaining: 45.9s
1600:	learn: 30.8206724	total: 48.9s	remaining: 42.7s
1700:	learn: 29.8608240	total: 52s	remaining: 39.7s
1800:	learn: 29.0783890	total: 55s

TypeError: got an unexpected keyword argument 'squared'

In [6]:
best_params = {'iterations': 3000, 'depth': 8, 'learning_rate': 0.2987646259036188, 'l2_leaf_reg': 0.018949650626008688, 'random_strength': 16.993667922363702, 'bagging_temperature': 0.9398476987366979}


# ----------------- Train final model on full data with best params -----------------
final_params = dict(best_params)
# ensure a few params are set
final_params.update({
    "loss_function": "MAE",
    "random_seed": RANDOM_SEED,
    "verbose": VERBOSE,
    "task_type": "GPU",   # GPU HERE
    "devices": "0"
})

print("\nTraining final model on full dataset...")
final_model = CatBoostRegressor(**final_params)
final_model.fit(X, y, verbose=VERBOSE)

# ----------------- Evaluation on hold-out: last fold as pseudo-test -----------------
# We'll evaluate final model on the last split's test set (most recent portion)
splits = list(tscv.split(X))
train_idx, test_idx = splits[-1]
X_tr, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_tr, y_test = y.iloc[train_idx], y.iloc[test_idx]

pred_test = final_model.predict(X_test)
metrics = evaluate(y_test, pred_test)
print("\nEvaluation on most recent fold (pseudo-test):")
print(metrics)

# ----------------- Per-station performance summary -----------------
print("\nPer-station MAE summary (on full dataset predictions)...")
df_preds = df.copy()
df_preds["pred"] = final_model.predict(X)
per_station = df_preds.groupby("StationId").apply(lambda g: mean_absolute_error(g[TARGET_COL], g["pred"]))
per_station = per_station.sort_values()
print(per_station.head(10))
print("...")

# ----------------- Feature importance -----------------
# try:
#     fi = final_model.get_feature_importance(prettified=True)
#     print("\nTop features (CatBoost importance):")
#     print(fi.head(20))
#     # save plot
#     fi_df = pd.DataFrame({
#         "feature": final_model.feature_names_,
#         "importance": final_model.get_feature_importance()
#     })
#     fi_df = fi_df.sort_values("importance", ascending=False).head(30)
#     plt.figure(figsize=(8,10))
#     plt.barh(fi_df["feature"][::-1], fi_df["importance"][::-1])
#     plt.title("Feature importance (top 30)")
#     plt.tight_layout()
#     plt.savefig("feature_importance_top30.png", dpi=150)
#     print("Feature importance plot saved -> feature_importance_top30.png")
# except Exception as e:
#     print("Could not compute feature importance:", e)


# ----------------- Save model and features -----------------
print("\nSaving model and feature list...")
joblib.dump(final_model, MODEL_SAVE)
joblib.dump(feature_list, FEATURES_SAVE)
print(f"Model saved to: {MODEL_SAVE}")
print(f"Feature list saved to: {FEATURES_SAVE}")

# ----------------- Quick summary -----------------
print("\nTraining complete.")
print("Baseline CV MAE (mean):", np.mean(cv_maes))
print("Final evaluation MAE:", metrics["MAE"])



Training final model on full dataset...


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 103.1693995	total: 220ms	remaining: 11m
100:	learn: 92.9906493	total: 3.64s	remaining: 1m 44s
200:	learn: 83.8974972	total: 7.11s	remaining: 1m 39s
300:	learn: 76.0529192	total: 11.2s	remaining: 1m 40s
400:	learn: 69.4540351	total: 15.4s	remaining: 1m 39s
500:	learn: 63.7415371	total: 19.2s	remaining: 1m 35s
600:	learn: 58.7515997	total: 23.1s	remaining: 1m 32s
700:	learn: 54.3575819	total: 27.3s	remaining: 1m 29s
800:	learn: 50.4233743	total: 31.2s	remaining: 1m 25s
900:	learn: 46.8417259	total: 34.9s	remaining: 1m 21s
1000:	learn: 43.5383220	total: 38.5s	remaining: 1m 16s
1100:	learn: 40.4385333	total: 42.4s	remaining: 1m 13s
1200:	learn: 37.5931904	total: 46s	remaining: 1m 8s
1300:	learn: 35.2972791	total: 49.7s	remaining: 1m 4s
1400:	learn: 33.4677773	total: 53.4s	remaining: 1m
1500:	learn: 31.9973293	total: 56.9s	remaining: 56.8s
1600:	learn: 30.8206724	total: 1m	remaining: 52.8s
1700:	learn: 29.8608240	total: 1m 4s	remaining: 48.9s
1800:	learn: 29.0783890	total: 1m 7s	r

NameError: name 'cv_maes' is not defined